In [101]:
# detect STR in the reference first
with open('ref_practice_W_3_chr_1.txt','r') as f:
    f.readline() # skip the first line
    ref=''
    for line in f:
        ref=ref+line.strip()
ref='GCTGTCCATTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGAGCTGTTTA'
STRlist=[]
i=0
potentialSTR=''
for k in range(2,7):
    i=0
    while i<len(ref):
        potentialSTR=ref[i:i+k]
        copynumber=0
        if ref[i+k:i+6*k]==potentialSTR*5:
            if k==4 and potentialSTR[0:2]==potentialSTR[2:4]:
                i+=1
                continue
            if k==6 and potentialSTR[0:2]==potentialSTR[2:4]==potentialSTR[4:6]:
                i+=1
                continue
            if k==6 and potentialSTR[0:3]==potentialSTR[3:6]:
                i+=1
                continue
            copynumber=6
            j=i+6*k
            while True:
                if ref[j:j+k]==potentialSTR:
                    copynumber+=1
                    j+=k
                else:
                    STRlist.append([potentialSTR*copynumber,i,copynumber])
                    break
            i=i+k*copynumber+1  
        else:
            i+=1
STRlist

[['TGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTG',
  9,
  49]]

In [95]:
#determine the copy number of STR: Can only determine total length <44 say. (read length =50, and need 3 nucleaotdies at both ends)
# collect all reads that aligned 3-20 nucleotides before 9561. 
#determine the STR for each read, and take the longest one. 
import pickle
(aligned_reads, alignments)=pickle.load(open('practice_W_3aligned_reads_and_coordinates','rb'))
STRinRef=pickle.load(open('practice_W_3_STR','rb'))
def determine_STR_inRef(ref):
    STRlist=[]
    i=0
    potentialSTR=''
    for k in range(2,7):
        i=0
        while i<len(ref):
            potentialSTR=ref[i:i+k]
            copynumber=0
            if ref[i+k:i+6*k]==potentialSTR*5:
                copynumber=6
                j=i+6*k
                while True:
                    if ref[j:j+k]==potentialSTR:
                        copynumber+=1
                        j+=k
                    else:
                        STRlist.append(['STR',potentialSTR*copynumber,i,copynumber])
                        break
                i=i+k*copynumber+1  
            else:
                i+=1
    STRlist = sorted(STRlist, key=lambda STR: STR[-2])
    return STRlist

def determine_STR(STRinRef, aligned_reads, alignments):
    import numpy as np
    STRlist=[]
    for STR in STRinRef:
        if len(STR[1])<44:
            reads=[]
            for i in range(len(alignments)):
                if STR[2]-20<alignments[i][0] <STR[2]-5:
                    reads.append(aligned_reads[i][0])
                if STR[2]-20<alignments[i][1] <STR[2]-5:
                    reads.append(aligned_reads[i][1])
            copynumber=[]
            STRunit=''
            for read in reads:
                L=determine_STR_inRef(read)
                if len(L)>0:
                    copynumber.append(L[0][-1])
                    unitlength=int(len(L[0][-3])/L[0][-1])
                    STRunit=L[0][-3][0:unitlength]
            cn=np.array(copynumber)
            determined_copynumber=cn.max()
            STRlist.append(['STR',STRunit*determined_copynumber,STR[2],determined_copynumber])
        else:
            STRlist.append(STR)
    return STRlist
determine_STR(STRinRef,aligned_reads, alignments)
#STRinRef
#determine_STR_inRef(read)

[['STR', 'GATGATGATGATGATGAT', 1787, 6],
 ['STR',
  'ATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTGATTTG',
  2286,
  19],
 ['STR', 'GCGCAGCGCAGCGCAGCGCAGCGCAGCGCAGCGCAGCGCAGCGCAGCGCAGCGCA', 6085, 11],
 ['STR', 'CAACAACAACAACAACAACAACAACAACAACAA', 9561, 11]]